### AutoGen Agents State Management 

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.base import TaskResult
from autogen_core import CancellationToken

import os

from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

if not all([azure_openai_api_key, azure_openai_endpoint, azure_openai_deployment_name, azure_openai_api_version]):
    raise ValueError("""
        Please set all required environment variables: 
            AZURE_OPENAI_API_KEY, 
            AZURE_OPENAI_ENDPOINT, 
            AZURE_OPENAI_DEPLOYMENT_NAME, 
            AZURE_OPENAI_API_VERSION
        """)


In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [ ]:
assistant_agent = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are a fun fact expert about animals. ",
)

response = await assistant_agent.on_messages(
    [
        TextMessage(
            content="Tell me a fun fact about Dolphins.",
            source="user",
        )
    ],
    cancellation_token=CancellationToken(),
)

print(response.chat_message.content)

In [ ]:
agent_state = await assistant_agent.save_state()

print("Agent state saved.")
print(agent_state)

In [ ]:
new_assistant_agent = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are a fun fact expert about animals. ",
)

await new_assistant_agent.load_state(agent_state)

response = await new_assistant_agent.on_messages(
    [
        TextMessage(
            content="What was the fun fact you told me about Dolphins?",
            source="user",
        )
    ],
    cancellation_token=CancellationToken(),
)

print(response.chat_message.content)